In [3]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append("../../src")
import pandas as pd
import numpy as np
import synapseclient as sc
import utils.munging_utils as query

In [65]:
df = query.get_walking_synapse_table(recordIds = ["db3d35fe-4798-462d-ab01-eb7bb47b75ff"],
                               table_id = "syn12514611",
                               version = "V2")

Querying V2 Data


In [66]:
def get_fs(path):
    data = query.open_filepath(path)[1:]
    data = data[data["sensorType"] == "acceleration"]
    time_end = (data["timestamp"][-1:].iloc[0])
    N = data.shape[0]
    return N/time_end

In [75]:
data = query.open_filepath(df["walk_motion.json_pathfile"].iloc[0])

In [82]:
data

,eventAccuracy,referenceCoordinate,sensorType,stepPath,timestamp,timestampDate,uptime,w,x,y,z
0,NaN,NaN,NaN,WalkAndBalance/walk/motion,0.000000,2019-11-21T13:44:22.487-08:00,877289.705068,NaN,NaN,NaN,NaN
1,NaN,NaN,accelerometer,WalkAndBalance/walk/motion,0.033036,NaN,877289.738104,NaN,0.010666,0.018173,-1.004013
2,NaN,NaN,accelerometer,WalkAndBalance/walk/motion,0.042741,NaN,877289.747809,NaN,-0.010437,0.014938,-1.003983
3,NaN,NaN,accelerometer,WalkAndBalance/walk/motion,0.052493,NaN,877289.757561,NaN,0.017380,0.024124,-0.999817
4,NaN,NaN,accelerometer,WalkAndBalance/walk/motion,0.062197,NaN,877289.767265,NaN,0.025970,0.026764,-0.997849
5,NaN,NaN,accelerometer,WalkAndBalance/walk/motion,0.071933,NaN,877289.777001,NaN,-0.019882,0.009415,-1.005234
6,NaN,NaN,accelerometer,WalkAndBalance/walk/motion,0.081668,NaN,877289.786736,NaN,-0.013794,0.012222,-1.005325
7,NaN,NaN,accelerometer,WalkAndBalance/walk/motion,0.091402,NaN,877289.796470,NaN,0.021851,0.027588,-1.004501
8,NaN,NaN,accelerometer,WalkAndBalance/walk/motion,0.101144,NaN,877289.806212,NaN,0.027649,0.024643,-0.999588
9,NaN,NaN,accelerometer,WalkAndBalance/walk/motion,0.110872,NaN,877289.815940,NaN,-0.012970,0.007858,-1.003281
